##                  House Prices : Advanced Regression Techniques

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#### Aim: Predict the sale price of a house

***

#  Import necessary libraries
-----------------------------------------------------------

In [ ]:

import pandas as pd

import sys 

import numpy as np

import seaborn as sns

from math import sqrt

from pylab import rcParams



from sklearn import metrics

from sklearn.metrics import mean_squared_error

from sklearn import linear_model

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import cross_val_score

from sklearn.model_selection import GridSearchCV



import sklearn

from sklearn.linear_model import LinearRegression

from sklearn.linear_model import ElasticNet, Lasso

from sklearn.svm import SVR

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

from sklearn.kernel_ridge import KernelRidge



from sklearn.ensemble import StackingRegressor



from sklearn.pipeline import make_pipeline

from sklearn.preprocessing import RobustScaler

from sklearn.model_selection import KFold, cross_val_score, train_test_split

from sklearn.metrics import mean_squared_error

from sklearn.metrics import accuracy_score

from sklearn.preprocessing import StandardScaler



%matplotlib inline

import matplotlib as mpl

import matplotlib.pyplot as plt

from matplotlib.colors import ListedColormap

import warnings
warnings.filterwarnings('ignore')

***

**------------------------------------------------------------ 1. LOADING & LOOKING AT THE DATA --------------------------------------------------------------**

- The housing dataset is available on Kaggle under “House Prices: Advanced Regression Techniques”. The “train.csv” file contains the training data and “test.csv” contains the testing data. The training data contains data for 1460 rows which corresponds to 1460 house’s data and   80 columns which correspond to the feature of those houses. Similarly, the testing data contains data of 1461 houses and their 79 attributes. 

In [ ]:
# load dataset 
df_train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')  

df_test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')  

In [ ]:
# check shape

print(df_train.shape)

print(df_test.shape)

In [ ]:
# look a first 10 rows of training data

df_train.head(10)

In [ ]:
# look a first 10 rows of testing data

df_test.head(10)

In [ ]:
# see all the column names

df_train.columns

In [ ]:
df_train.info()

- There are 1460 rows and 81 columns

- There are columns with large number of null entries like PoolQC, MiscFeature

- The columns have Three types of datatypes: float64(3), int64(35), object(43)

In [ ]:
df_test.info()

- There are 1459 rows and 80 columns

- There are columns with large number of null entries like PoolQC, MiscFeature etc

- The columns have Three types of datatypes:  float64(11), int64(26), object(43)

#### Looking at the label to predict 

In [ ]:
df_train['SalePrice'].describe()

- The average SalePrice of a house is 180,921

- The Maximum SalePrice of a house is 755,000 and Minimum 34,900

In [ ]:
# Select only numeric columns from the DataFrame
numeric_df = df_train.select_dtypes(include=['float64', 'int64'])

# Compute the correlation matrix
corr_mat = numeric_df.corr()

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(12, 9))

# Draw the heatmap
sns.heatmap(corr_mat, vmax=.8, square=True)

# Set the title and labels
plt.suptitle("Correlation Feature Heatmap")
plt.xlabel("Features")
plt.ylabel("Features")

# Show the plot
plt.show()


In [ ]:
# Compute the correlation matrix for numeric columns
corr_mat = df_train.select_dtypes(include=['float64', 'int64']).corr()

# Set the font scale and figure size for the heatmap
sns.set(font_scale=1.3)
plt.figure(figsize=(11, 8))

# Select features with correlation higher than 0.5 with 'SalePrice'
top_corr = corr_mat.index[abs(corr_mat["SalePrice"]) > 0.5]

# Plot heatmap for top correlated features
g = sns.heatmap(df_train[top_corr].corr(), annot=True, cmap="YlGnBu")

# Set title and labels
plt.suptitle("Top Correlated Feature Heatmap (Correlation > 0.5 with SalePrice)")
plt.xlabel("Features")
plt.ylabel("Features")

# Show the plot
plt.show()


- OverallQual and GrLivArea seem to be the most correlated to SalePrice

In [ ]:
# Compute the correlation matrix for numeric columns
numeric_df = df_train.select_dtypes(include=['float64', 'int64'])

# Drop 'SalePrice' from the correlation matrix
corr = numeric_df.corr().drop('SalePrice')

# Sort the correlation values with 'SalePrice' in descending order
corr_sorted = corr['SalePrice'].sort_values(ascending=False)

# Print sorted correlation values
print("Correlation Values")
print(corr_sorted)


In [ ]:
rcParams['figure.figsize'] = 5,5

cols = ['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars','GarageArea', 'TotalBsmtSF','1stFlrSF','FullBath','YearBuilt']

sns_plot = sns.pairplot(df_train[cols])



plt.suptitle('Scatter plots between top 9 most corr features', y=1.04, size=25)

plt.tight_layout()

plt.show()

In [ ]:
rcParams['figure.figsize'] = 5,5

cols = ['SalePrice','EnclosedPorch', 'KitchenAbvGr', 'MSSubClass', 'LowQualFinSF','YrSold', 'OverallCond']

sns_plot = sns.pairplot(df_train[cols])



plt.suptitle('Scatter plots between least 6  corr features', y=1.04, size=20)

plt.tight_layout()

plt.show()

#### -------------------------------------------------------------------- 2. HANDLING DATA --------------------------------------------------------------------

#### Drop Id Column

In [ ]:
#drop id as it is not required for training or prediction

train_ID = df_train['Id']

test_ID = df_test['Id']



df_train.drop(['Id'], axis=1, inplace=True)

df_test.drop(['Id'], axis=1, inplace=True)



df_train.shape, df_test.shape

#### Checking for Outliers

In [ ]:

# Set the style for the plots
sns.set_style('whitegrid')
edgecolor = 'black'

# Function to plot scatter plot between a feature and the Sale Price 
def scatter_plot(a):
    # Create a new figure for each scatter plot
    plt.figure(figsize=(12, 12))
    
    plt.scatter(x=df_train[a], y=df_train['SalePrice'], edgecolor=edgecolor)
    plt.ylabel('SalePrice', fontsize=12)
    plt.xlabel(a, fontsize=12)
    plt.suptitle("Scatter Plot of " + a + " and Sale Price", fontsize=14)
    plt.show()

# Example call to the function
scatter_plot('LotArea')  # Replace 'LotArea' with any feature you want to plot

In [ ]:
scatter_plot('GrLivArea')



- It can be observed that there are large outliers which can negatively affect the prediction of sale price highly

- So the outliers need to be deleted

In [ ]:
#Deleting outliers

df_train =  df_train.drop( df_train[( df_train['GrLivArea'] > 4000) & ( df_train['SalePrice']<300000)].index)



#Check the graphic again

scatter_plot('GrLivArea')

In [ ]:
scatter_plot('TotalBsmtSF')

- There arent too large outliers, we do not need to delete any points

In [ ]:
scatter_plot('EnclosedPorch')

- There is are some outliers that should be deleted so that it doesnt affect our predictions much

In [ ]:
#Deleting outliers

df_train =  df_train.drop( df_train[( df_train['EnclosedPorch']>400)].index)



#Deleting outliers

df_train =  df_train.drop( df_train[( df_train['SalePrice']>700000)].index)



#check plot again

scatter_plot('EnclosedPorch')

In [ ]:
# plot a box plot for categorical feature : Overall Quality



fig = plt.figure(figsize=(7,7))

data = pd.concat([df_train['SalePrice'], df_train['OverallQual']], axis=1)

sns.boxplot(x = df_train['OverallQual'], y="SalePrice", data = data)

In [ ]:
# plot a box plot for categorical feature : Year Built

fig = plt.figure(figsize=(18,8))



data = pd.concat([df_train['SalePrice'], df_train['YearBuilt']], axis=1)

sns.boxplot(x= df_train['YearBuilt'], y="SalePrice", data=data)

plt.xticks(rotation=90,fontsize= 9)

In [ ]:
sns.distplot(df_train['SalePrice'])



plt.suptitle( "Plot of Sale Price")



print("Skewness: %f" % df_train['SalePrice'].skew())

print("Kurtosis: %f" % df_train['SalePrice'].kurt())

In [ ]:
# applying log transformation to correct the positive skewness in the data

# taking logs means that errors in predicting expensive and cheap houses will affect the result equally



df_train['SalePrice'] = np.log(df_train['SalePrice'])

plt.suptitle("Plot of Sale Price after log transformation")

sns.distplot(df_train['SalePrice'])

plt.show()

In [ ]:
df_train['SalePrice'].describe()

In [ ]:
df_train['SalePrice']

In [ ]:
df_train.shape

#### Handling missing data

In [ ]:
#function to see the missing data in a dataframe

def missing_data(df,n):    

    total = df.isnull().sum().sort_values(ascending=False)          # Total No of missing values

    percentage = (df.isnull().sum() / df.isnull().count()).sort_values(ascending=False)*100  # % of Missing values

    No_unique_val = df.nunique()                                   # No of unique values

    missing_data = pd.concat([total, percentage, No_unique_val], axis=1, 

                             keys=['Total No of missing val', '% of Missing val','No of unique val'], sort = False)

    

    print(missing_data.head(n))

In [ ]:
#training data    

missing_data(df_train,20)

In [ ]:
df_train['PoolQC'].unique()

- PoolQC,Alley have only two unique values

- PoolQC has 99.7% of missing data, which means most of the values are NA: No Pool ie most of the houses do not have a pool

- PoolQC,Alley,MiscFeature will be dropped due to large number of missing values

In [ ]:
#test data 

missing_data(df_test,34)

In [ ]:
df_test['Utilities'].unique()

- all records mostly "AllPub" for Utilities

- PoolQC,Alley,MiscFeature will be dropped due to large number of missing values

- Utilities has only 1 unique value

- Utility will also be dropped

In [ ]:
# calculate total number of null values in training data

null_train = df_train.isnull().sum().sum()

print(null_train)



# calculate total number of null values in test data

null_test = df_test.isnull().sum().sum()

print(null_test)

In [ ]:
# save the 'SalePrice'column as train_label

train_label = df_train['SalePrice'].reset_index(drop=True)



# # drop 'SalePrice' column from df_train 

df_train = df_train.drop(['SalePrice'], axis=1)

# # now df_train contains all training features

In [ ]:
# function to HANDLE the missing data in a dataframe

def missing (df):

    

    # drop theses columns due to large null values or many same values

    df = df.drop(['Utilities','PoolQC','MiscFeature','Alley'], axis=1)

    

    # Null value likely means No Fence so fill as "None"

    df["Fence"] = df["Fence"].fillna("None") 

    

    # Null value likely means No Fireplace so fill as "None"

    df["FireplaceQu"] = df["FireplaceQu"].fillna("None")

    

    # Lot frontage is the feet of street connected to property, which is likely similar to the neighbourhood houses, so fill Median value

    df["LotFrontage"] = df["LotFrontage"].fillna(df["LotFrontage"].median())

    

    # Null value likely means  typical(Typ)

    df["Functional"] = df["Functional"].fillna("Typ")

    

    # Only one null value so fill as the most frequent value(mode)

    df['KitchenQual'] = df['KitchenQual'].fillna(df['KitchenQual'].mode()[0])  

    

    # Only one null value so fill as the most frequent value(mode)

    df['Electrical'] = df['Electrical'].fillna(df['Electrical'].mode()[0])

    

    # Very few null value so fill with the most frequent value(mode)

    df['SaleType'] = df['SaleType'].fillna(df['SaleType'].mode()[0])

    

    # Null value likely means no masonry veneer

    df["MasVnrType"] = df["MasVnrType"].fillna("None") #so fill as "None" (since categorical feature)

    df["MasVnrArea"] = df["MasVnrArea"].fillna(0)      #so fill as o

    

    # Only one null value so fill as the most frequent value(mode)

    df['Exterior1st'] = df['Exterior1st'].fillna(df['Exterior1st'].mode()[0])

    df['Exterior2nd'] = df['Exterior2nd'].fillna(df['Exterior2nd'].mode()[0])

    

    #MSZoning is general zoning classification,Very few null value so fill with the most frequent value(mode)

    df['MSZoning'] = df['MSZoning'].fillna(df['MSZoning'].mode()[0])

    

    #Null value likely means no Identified type of dwelling so fill as "None"

    df['MSSubClass'] = df['MSSubClass'].fillna("None")

    

    # Null value likely means No Garage, so fill as "None" (since these are categorical features)

    for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):

        df[col] = df[col].fillna('None')

    

    # Null value likely means No Garage and no cars in garage, so fill as 0

    for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):

        df[col] = df[col].fillna(0)

    

    # Null value likely means No Basement, so fill as 0

    for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):

        df[col] = df[col].fillna(0)

    

    # Null value likely means No Basement, so fill as "None" (since these are categorical features)

    for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):

        df[col] = df[col].fillna('None')

    

    return df

In [ ]:
df_train = missing(df_train)

df_test = missing(df_test)

In [ ]:
# calculate total number of null values in training data

null_train = df_train.isnull().sum().sum()

print(null_train)



# calculate total number of null values in test data

null_test = df_test.isnull().sum().sum()

print(null_test)

In [ ]:
df_train.shape,df_test.shape

In [ ]:
def add_new_cols(df):

    

    df['Total_SF'] = df['TotalBsmtSF'] + df['1stFlrSF'] + df['2ndFlrSF']



    df['Total_Bathrooms'] = (df['FullBath'] + (0.5 * df['HalfBath']) + df['BsmtFullBath'] 

                             + (0.5 * df['BsmtHalfBath']))



    df['Total_Porch_SF'] = (df['OpenPorchSF'] + df['3SsnPorch'] + df['EnclosedPorch'] + 

                            df['ScreenPorch'] + df['WoodDeckSF'])



    df['Total_Square_Feet'] = (df['BsmtFinSF1'] + df['BsmtFinSF2'] + df['1stFlrSF'] + df['2ndFlrSF'])

    

    df['Total_Quality'] = df['OverallQual'] + df['OverallCond']

    

    return df

In [ ]:
# add the new columns

df_train = add_new_cols(df_train)

df_test = add_new_cols(df_test)

In [ ]:
df_train.shape,df_test.shape

#### Check data types

In [ ]:
#training data

g1 = df_train.columns.to_series().groupby(df_train.dtypes).groups

In [ ]:
{k.name: v for k, v in g1.items()}

In [ ]:
#testing data

g2 = df_test.columns.to_series().groupby(df_test.dtypes).groups

In [ ]:
{k.name: v for k, v in g2.items()}

In [ ]:
#get dummy values for categorical data

df_train = pd.get_dummies(df_train)

df_test = pd.get_dummies(df_test)



print(df_train.shape)

print(df_test.shape)

In [ ]:
#align the training and testing data

df_train, df_test = df_train.align(df_test, join = 'inner', axis=1)

In [ ]:
print(df_train.shape)

print(df_test.shape)

In [ ]:
# calculate total number of null values in training data

null_train = df_train.isnull().sum().sum()

print(null_train)



# calculate total number of null values in test data

null_test = df_test.isnull().sum().sum()

print(null_test)

In [ ]:
df_train.head(5)

In [ ]:
df_test.head(5)

In [ ]:
df_train.info()

In [ ]:
X_test = df_test           # testing features

In [ ]:
df_train["SalePrice"] = train_label

In [ ]:
df_train.head()

In [ ]:
train_set, valid_set = train_test_split(df_train,train_size= 0.7, shuffle=False)



X_train = train_set.drop(["SalePrice"], axis=1)  # training features

y_train = train_set["SalePrice"].copy()             # training label



X_valid = valid_set.drop(["SalePrice"], axis=1)  # testing features

y_valid = valid_set["SalePrice"].copy()               # testing label

In [ ]:
print("X_train shape: {}".format(X_train.shape))

print("y_train shape: {}".format(y_train.shape))

print()

print("X_valid shape: {}".format(X_valid.shape))

print("y_valid shape: {}".format(y_valid.shape))

print()

print("X_test shape: {}".format(X_test.shape))

#### Check data type and null values

In [ ]:
X_train.info()

In [ ]:
X_valid.info()

In [ ]:
y_train

In [ ]:
y_valid

In [ ]:
null_t_x = X_train.isnull().sum().sum()

print(null_t_x)



null_t_y = y_train.isnull().sum().sum()

print(null_t_y)

In [ ]:
null_v_x = X_valid.isnull().sum().sum()

print(null_v_x)



null_v_y = y_valid.isnull().sum().sum()

print(null_v_y)

- No null values in X_valid

- There are 5 null values in y_valid

In [ ]:
np.where(np.isnan(y_valid))

In [ ]:
# replace null values by mean value of y_valid column

mean = np.nanmean(y_valid)

y_valid = np.nan_to_num(y_valid,nan = mean)

In [ ]:
#check again

np.where(np.isnan(y_valid))

In [ ]:
y_valid.dtype

In [ ]:
print("Valid data shape:")

print(X_valid.shape, y_valid.shape)

print()

***

#### -------------------------------------------------------- 3. SET CROSS VALIDATION AND RMSE --------------------------------------------------

### Cross Validation



- done to avoid underfitting/overfitting of data and to get a better understanging of how good our models are performing

- split  data into k subsets, and train on k-1 of those subset,leaving one for testing

- performing 10-fold cross validation for each model#

In [ ]:
# calculating cross validation score with scoring set to negative mean absolute error

def cross_validation(model):

    

    scores = np.sqrt(-cross_val_score(model, X_train, y_train, cv = 12, scoring = "neg_mean_squared_error"))

    mean = np.mean(scores)

    print("Mean CV score: ",mean)

### RMSE

In [ ]:
# function to calculate Root mean square error (RMSE)

def rmse(y_pred, y_train): 

    

    rmse_ = np.sqrt(metrics.mean_squared_error(y_pred,y_train))

    print("rmse: ", rmse_)

### Plot Label

In [ ]:
# function to plot actual vs predicited label

def actual_vs_pred_plot(y_train,y_pred):

    

    fig = plt.figure(figsize=(12,12))

    fig, ax = plt.subplots()

    

    ax.scatter(y_train, y_pred,color = "teal",edgecolor = 'lightblue')

    ax.plot([y_train.min(),y_train.max()], [y_train.min(), y_train.max()], 'k--',lw=0.2)

    ax.set_xlabel('Actual')

    ax.set_ylabel('Predicted')

    plt.suptitle("Actual vs Predicted Scatter Plot",size=14)

    plt.show()

***

#### ---------------------------------------------------------------------- 4. DATA MODELLING  -------------------------------------------------------------------

### MODELS

#### 1. LINEAR REGRESSION MODEL

- Linear Regression is the first model used. In this model, the target value is expected to be a linear combination of the features. The coefficients are set to minimize the residual sum of squares between the target predicted and the observed features

In [ ]:
reg = linear_model.LinearRegression()

In [ ]:
cross_validation(reg)

In [ ]:
#fit on training

model_reg = reg.fit(X_train, y_train)



#predict value of sale price on the training set

y1_pred = reg.predict(X_train)



#caculate root mean square error

rmse(y1_pred,y_train)

In [ ]:
#predict value of sale price on the validation set

y1_pred_v = reg.predict(X_valid)



#caculate root mean square error

rmse(y1_pred_v, y_valid)

In [ ]:

#plot

actual_vs_pred_plot(y_valid,y1_pred_v)

#### 2. RIDGE MODEL

- The second model used is Ridge Regression. Ridge Regression is a regularized version of linear regression. The parameter alpha is used to regularize the model. For alpha equal to zero, ridge regression is just a linear regression. RidgeCV model is used to implement ridge regression as it has a built-in cross validation of the alpha parameter. Sixteen different values of alpha between 7e-4 and 20 were used with a 10-fold cross validation. A pipeline using min-max scaler was built to apply to training, validation and testing data. 

In [ ]:
# to find the best value of alphas from this list, i will use RidgeCV

alphas_ = [ 7e-4, 5e-4, 3e-4, 1e-4, 1e-3, 5e-2, 1e-2, 0.1, 0.3, 1, 3, 5, 10, 15, 18, 20]



# use robust scaler as unlike other scalers, the centering and scaling of ro bust scaler

#is based on percentiles and are therefore is not influenced by a few number of very large marginal outliers.



ridge = make_pipeline(MinMaxScaler(), linear_model.RidgeCV(alphas = alphas_, cv = 10))

In [ ]:
cross_validation(ridge)

In [ ]:
#fit

model_ridge = ridge.fit(X_train, y_train)



#predict value of sale price on the training set

y2_pred = ridge.predict(X_train)



#caculate root mean square error

rmse(y2_pred,y_train)

In [ ]:
#predict value of sale price on the valid set

y2_pred_v = ridge.predict(X_valid)



#caculate root mean square error

rmse(y2_pred_v, y_valid)

In [ ]:
#plot

actual_vs_pred_plot(y_train,y2_pred)

#### 3. LASSO MODEL


- Lasso regression is also a regularized version of linear regression. Lasso regression automatically performs feature selection and can estimates sparse coefficients.  LassoCV model was used to implement lasso regression as it has a built-in cross validation of the alpha parameter. Different values of alpha were set with a 10-fold cross validation. Robust scaler was used in a pipeline to scale the training, validation and testing data. 

In [ ]:
# to find the best value of alphas from this list, i will use LassoCV

alpha2 = [0.0001, 0.0002, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008]



#use robust scaler so that predictions are not influenced by a few number of very large marginal outliers



lasso = make_pipeline(RobustScaler(), linear_model.LassoCV(alphas = alpha2, random_state=42,cv=12,max_iter=2000))

In [ ]:
cross_validation(lasso)

In [ ]:
#fit

model_lasso = lasso.fit(X_train, y_train)



#predict value of quality on the training set

y3_pred = lasso.predict(X_train)



#caculate root mean square error

rmse(y3_pred,y_train)

In [ ]:
#predict value of sale price on the validation set

y3_pred_v = lasso.predict(X_valid)



#caculate root mean square error

rmse(y3_pred_v, y_valid)

In [ ]:
actual_vs_pred_plot(y_valid,y3_pred_v)

#### 4. K-NEAREST NEIGHBOUR REGRESSION MODEL


- K -nearest neighbour regressor is another popular model for regression tasks. It is a simple supervised machine learning model. The numbers of neighbours were set to three different values and the performance of this model was noted. Weights were set to uniform to assign equal weights to all points in each neighbourhood. The algorithm used was set to auto so that the best performing algorithm on the values was used. The leaf size was set to 25.

In [ ]:
from sklearn.neighbors import KNeighborsRegressor



# N = 5 #

neigh = KNeighborsRegressor(n_neighbors = 5,

                            weights = 'uniform',

                            algorithm = 'auto',

                            leaf_size=25)

neigh.fit(X_train,y_train)



#predict value of sale price on the training set

y4_pred = neigh.predict(X_train)



#caculate root mean square error

rmse(y4_pred,y_train)

In [ ]:
# N = 7 #

neigh1 = KNeighborsRegressor(n_neighbors = 7,

                             weights = 'uniform',

                             leaf_size=25)

neigh1.fit(X_train,y_train)



#predict value of quality on the training set

y_pred = neigh1.predict(X_train)



#caculate root mean square error

rmse(y_pred,y_train)

In [ ]:
# N = 9 #

neigh2 = KNeighborsRegressor(n_neighbors = 9,

                             weights = 'uniform',

                             leaf_size=25)

neigh2.fit(X_train,y_train)



#predict value of quality on the training set

y_pred = neigh2.predict(X_train)



#caculate root mean square error

rmse(y_pred,y_train)

In [ ]:
# N=5 performs best

In [ ]:
#predict value of sale price on the validation set

y4_pred_v = neigh.predict(X_valid)



#caculate root mean square error

rmse(y4_pred_v, y_valid)

Note: rmse increases when values of k(no. of neighbours) increase 

In [ ]:
actual_vs_pred_plot(y_valid,y4_pred_v)

#### 5. DECISION TREE MODEL

- Decision tree model is also used to fit this data as it does not require much data cleaning and is not influenced by outliers. Decision trees can, unlike linear models, fit linearly inseparable datasets. The values of minimum leaves were set between 1 to 9 because a very small number of minimum leaves can cause overfitting whereas a large number of minimum leaves will prevent the tree from learning. Maximum depth of 7 and 9 were used to fit the data for predictions. 

In [ ]:
from sklearn import tree

In [ ]:
# set max depth to 5

tree_regr1 = tree.DecisionTreeRegressor(max_depth = 7, min_samples_leaf=5,random_state=42)



# set max depth to 9

tree_regr2 = tree.DecisionTreeRegressor(max_depth = 9,min_samples_leaf=9,random_state=42)



#fit the traning data to a decision tree model

tree_regr11 = tree_regr1.fit(X_train,y_train)

tree_regr12 = tree_regr2.fit(X_train,y_train)



#predict value of sale price on the training set

y1 = tree_regr1.predict(X_train)

y2 = tree_regr2.predict(X_train)

In [ ]:
cross_validation(tree_regr1)

cross_validation(tree_regr2)

In [ ]:
#caculate root mean square error

rmse(y1,y_train)

In [ ]:
rmse(y2,y_train)

In [ ]:
#predict value of sale price on the validation set

y5_pred_v = tree_regr2.predict(X_valid)



#caculate root mean square error

rmse(y5_pred_v, y_valid)

In [ ]:
#plot

actual_vs_pred_plot(y_valid,y5_pred_v)

#### 6. Random Forest MODEL

- Random forest model is an ensemble method based on randomized decision trees. Grid search was used to select the best parameters with a 5-fold cross validation. The number of trees in the forest was set to 200 with a maximum depth of 5 and 3 minimum leaves. 

In [ ]:
rforest = RandomForestRegressor(n_estimators=200,max_depth=13,random_state=42)

In [ ]:
# grid search to find best value of C, gamma and epsilon

param_grid  = {'n_estimators': [100,150,200,250,300,350,400],

               'max_depth': [5,7,9,11,13,15,17], 

               'min_samples_leaf': [3,5,7,9,11,13,15]}



# set cross validation to 5

clf = GridSearchCV(rforest, param_grid, cv = 5, n_jobs = -2)

clf.fit(X_train,y_train)

In [ ]:
clf.best_params_

In [ ]:
rforest = RandomForestRegressor(n_estimators=, max_depth=5, min_samples_leaf=3, random_state=42)

In [ ]:
cross_validation(rforest)

In [ ]:
#fit

model_rforest = rforest.fit(X_train, y_train)



#predict value of sale price on the training set

y6_pred = rforest.predict(X_train)



#caculate root mean square error

rmse(y6_pred,y_train)

In [ ]:
#predict value of sale price on the validation set

y6_pred_v = rforest.predict(X_valid)



#caculate root mean square error

rmse(y6_pred_v, y_valid)

In [ ]:
#0: 0.38852359192540425

#1: 0.38616747296757176

In [ ]:
#plot

actual_vs_pred_plot(y_valid, y6_pred_v)

#### 7. Support Vector Regressor MODEL

- Support vector regressor is another powerful model. It is memory efficient and offers different kernels to choose from. Grid search was used to find the best value of the hyperparameters C, gamma and epsilon. The sigmoid kernel was used along with the default value of epsilon. 

In [ ]:
svr_basic = SVR(C = 10, gamma = 0.001)

In [ ]:
# grid search to find best value of C, gamma and epsilon and default kernel 'rbf'

param_grid  = {'C': [5,7,10,15,20,30],'gamma': [0.001, 0.0001, 0.0011, 0.00011], 'epsilon': [0.1, 0.01, 0.001, 0.005, 0.007, 0.008, 0.009] }



# set cross validation to 5

clf = GridSearchCV(svr_basic, param_grid, cv = 10, n_jobs = -2)

clf.fit(X_train,y_train)

In [ ]:
clf.best_params_

In [ ]:
#make final SVR model with best parameters found from grid search

svr = make_pipeline(MinMaxScaler(), SVR(C= 5, epsilon= 0.1, gamma=0.0011, kernel = "sigmoid"))

In [ ]:
cross_validation(svr)

In [ ]:
#fit

model_svr = svr.fit(X_train, y_train)



#predict value of sale price on the training set

y7_pred = svr.predict(X_train)



#caculate root mean square error

rmse(y7_pred,y_train)

In [ ]:
#predict value of sale price on the validation set

y7_pred_v = svr.predict(X_valid)



#caculate root mean square error

rmse(y7_pred_v, y_valid)

In [ ]:
# Linear - 0.4338387095039476

# Sigmoid - 0.3900469727418305

# With sigmoid as default kernel - 0.39670545624904924

# rbf - 0.39420253052849114

In [ ]:
actual_vs_pred_plot(y_valid, y7_pred_v)

#### 8. Gradient Boosting Regressor MODEL

- Gradient boosting regression is an ensemble of weak prediction models. Two gradient boosting models with different depths were evaluated. The loss was set to ‘huber’ which is a combination of least square regression and a highly robust loss function. 

In [ ]:
# set max depth to 4, min_samples_leaf to 15

gbr1 = GradientBoostingRegressor(n_estimators=200, learning_rate=0.05, max_depth = 7,

                                min_samples_leaf=7, loss='huber', random_state =42) 

In [ ]:
# set max depth to 7, min_samples_leaf to 10

gbr2 = GradientBoostingRegressor(n_estimators=200, learning_rate=0.05, max_depth = 9,

                                min_samples_leaf=10, loss='huber', random_state =42) 

In [ ]:
cross_validation(gbr1)

cross_validation(gbr2)

In [ ]:
#fit

model_gbr1 = gbr1.fit(X_train, y_train)

model_gbr2 = gbr2.fit(X_train, y_train)



#predict value of sale price on the training set

y_g1_pred = gbr1.predict(X_train)

y_g2_pred = gbr2.predict(X_train)



#caculate root mean square error

rmse(y_g1_pred,y_train)

rmse(y_g2_pred,y_train)

- model gbr2 performs best

In [ ]:
#predict value of sale price on the validation set

y8_pred_v = gbr2.predict(X_valid)



#caculate root mean square error

rmse(y8_pred_v, y_valid)

In [ ]:
# plot for gbr2

actual_vs_pred_plot(y_valid, y8_pred_v)

#### 9. STACKED REGRESSOR MODEL

- The final model used is the stacked regressor model. Stacking allows the power of each individual estimator to be used by using their output as a final estimator input. Random forest, Support vector regressor, K -nearest neighbour regressor and ridge regressor were stacked with random forest as the final estimator.

In [ ]:
# using Random Forest,Support Vector Regressor and Gradient Boosting to build a stack model because they have lower RMSE comparatively

estimators = [('Random Forest', rforest),

              ("Support Vector Regressor",svr),

              ("K",neigh),

              ("Ridge",ridge)

              ]

In [ ]:
stacked = StackingRegressor(estimators = estimators, final_estimator = rforest, cv=5)

In [ ]:
cross_validation(stacked)

In [ ]:
#fit

model_stack = stacked.fit(X_train, y_train)



#predict value of sale price on the training set

y9_pred = stacked.predict(X_train)



#caculate root mean square error

rmse(y9_pred,y_train)

In [ ]:
#predict value of sale price on the validation set

y9_pred_v = stacked.predict(X_valid)



#caculate root mean square error

rmse(y9_pred_v, y_valid)

In [ ]:
# plot

actual_vs_pred_plot(y_valid,y9_pred_v)

### Observations

#### RMSE:

- linear reg                        : 0.42793480397157035

- ridge                             : 0.3957886167433282

- lasso                             : 0.4059493256188701

- k-nearest neighbour(k=5)          : 0.41351487769327555

- decision tree(maxdepth=9)         : 0.4583579345988703

- random forest                     : 0.38616747296757176

- Support Vector Regressor          : 0.3900469727418305

- Gradient Boosting Regressor       : 0.4118219430457788

- Stacked Regressor model           : 0.3769718491202983

#### How errors compare:

- The lowest error is of  : Stacked Regressor model  

- The largest error is of : decision tree(maxdepth=9)

- Therefore Stacked Regressor model will be applied to the test data as it is the best performing model

#### -------------------------------------------------------------------- 5. TEST DATA PREDICTION -----------------------------------------------------------------

In [ ]:
csv_path = "sample_submission.csv"

df_sub = pd.read_csv(csv_path, sep = ',')  

In [ ]:
df_sub.shape

In [ ]:
df_sub.head()

In [ ]:
X_test.shape

In [ ]:
#predict value of sale price on the training set

y_final_pred = stacked.predict(X_test)



y_final_pred

In [ ]:
#undo the log tranformation to get predictions in terms of original label

predictions = np.expm1(y_final_pred)

print(predictions)

In [ ]:
submit = pd.DataFrame()

submit['Id'] = test_ID

submit['SalePrice'] = predictions

submit.to_csv('submission.csv',index=False)

In [ ]:
submit

***